# Using the openrefine-client in a Python 2 environment

## Preparations

First we need an OpenRefine server running and the openrefine-client installed.

### Option 1: binder

This [binder](https://github.com/betatim/openrefineder) has OpenRefine, the openrefine-client and a Jupyter server proxy preinstalled. We need to start the OpenRefine server proxy by opening the urlpath `/openrefine`. It is a bit complicated doing it directly from this notebook but the following commands will do that for you. This may take up to 30 seconds.

In [1]:
import os
if 'openrefineder' in os.environ['HOSTNAME']:
    notebook = !jupyter notebook list | grep -o -E 'http\S+'
    openrefine_url = notebook[0].replace('?token', 'openrefine?token')
    from urllib import urlopen
    from time import sleep
    for i in range(30):
        response = urlopen(openrefine_url).read()
        sleep(1)
        if 'openrefine' in response:
            openrefine_url = openrefine_url.replace('http://0.0.0.0:8888','')
            from IPython.core.display import display, HTML
            display(HTML('<a href="' + openrefine_url + '">Click here to open OpenRefine</a>'))
            break
        if i == 30:
            print('timeout')

### Option 2: Local environment

Ensure you have an OpenRefine server running. Then install the OpenRefine client as follows.

```
pip install openrefine-client
```

## Create a directory

We will store some files so it is clearer to use a new folder.

In [2]:
import os, datetime
path = os.path.expanduser('~') + '/' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
try:
    os.mkdir(path)
    os.chdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print (os.getcwd())

/home/jovyan/20190823_154057


## Import module

In [3]:
from google.refine import cli

## Create project

Download sample data

In [4]:
cli.download('https://git.io/fj5hF','duplicates.csv')

Download to file duplicates.csv complete


Import file into OpenRefine (and store returned project)

In [5]:
p1 = cli.create('duplicates.csv')

id: 1912905797111
rows: 10


## List all projects

In [6]:
cli.ls()

 1912905797111: duplicates


## Show project metadata

In [7]:
cli.info(p1.project_id)

                  id: 1912905797111
                 url: http://127.0.0.1:3333/project?project=1912905797111
                name: duplicates
            modified: 2019-08-23T15:40:58Z
             created: 2019-08-23T15:40:58Z
            rowCount: 10
importOptionMetadata: [{u'storeEmptyStrings': True, u'fileSource': u'duplicates.csv', u'storeBlankRows': True, u'encoding': u'', u'projectName': u'duplicates', u'processQuotes': True, u'separator': u',', u'trimStrings': False, u'limit': -1, u'storeBlankCellsAsNulls': True, u'guessCellValueTypes': False, u'includeFileSources': False}]
          column 001: email
          column 002: name
          column 003: state
          column 004: gender
          column 005: purchase


## Export project to terminal

In [8]:
cli.export(p1.project_id)

email	name	state	gender	purchase
danny.baron@example1.com	Danny Baron	CA	M	TV
melanie.white@example2.edu	Melanie White	NC	F	iPhone
danny.baron@example1.com	D. Baron	CA	M	Winter jacket
ben.tyler@example3.org	Ben Tyler	NV	M	Flashlight
arthur.duff@example4.com	Arthur Duff	OR	M	Dining table
danny.baron@example1.com	Daniel Baron	CA	M	Bike
jean.griffith@example5.org	Jean Griffith	WA	F	Power drill
melanie.white@example2.edu	Melanie White	NC	F	iPad
ben.morisson@example6.org	Ben Morisson	FL	M	Amplifier
arthur.duff@example4.com	Arthur Duff	OR	M	Night table


## Apply rules from json file

Download sample json file (the content of this file was previously extracted via Undo/Redo history in the OpenRefine graphical user interface)

In [9]:
cli.download('https://git.io/fj5ju','duplicates-deletion.json')

Download to file duplicates-deletion.json complete


Apply transformations rules

In [10]:
cli.apply(p1.project_id, 'duplicates-deletion.json')

File duplicates-deletion.json has been successfully applied to project 1912905797111


Export project to terminal again

In [11]:
cli.export(p1.project_id)

email	count	name	state	gender	purchase
arthur.duff@example4.com	2	Arthur Duff	OR	M	Dining table
ben.morisson@example6.org	1	Ben Morisson	FL	M	Amplifier
ben.tyler@example3.org	1	Ben Tyler	NV	M	Flashlight
danny.baron@example1.com	3	Danny Baron	CA	M	TV
jean.griffith@example5.org	1	Jean Griffith	WA	F	Power drill
melanie.white@example2.edu	2	Melanie White	NC	F	iPhone


## Export project to file

Export data in Excel (.xls) format

In [12]:
cli.export(p1.project_id, 'deduped.xls')

email	count	name	state	gender	purchase
arthur.duff@example4.com	2	Arthur Duff	OR	M	Dining table
ben.morisson@example6.org	1	Ben Morisson	FL	M	Amplifier
ben.tyler@example3.org	1	Ben Tyler	NV	M	Flashlight
danny.baron@example1.com	3	Danny Baron	CA	M	TV
jean.griffith@example5.org	1	Jean Griffith	WA	F	Power drill
melanie.white@example2.edu	2	Melanie White	NC	F	iPhone


## Delete project

In [13]:
cli.delete(p1.project_id)

Project 1912905797111 has been successfully deleted


## Advanced templating

Create another project from the example file above

In [14]:
p2 = cli.create('duplicates.csv')

id: 2004697124065
rows: 10


The following example code will export the columns "name" and "purchase" in JSON format from the project "advanced" for rows matching the regex text filter ^F$ in column "gender"

In [15]:
cli.templating(p2.project_id,
prefix='''{ "events" : [
''',
template='    { "name" : {{jsonize(cells["name"].value)}}, "purchase" : {{jsonize(cells["purchase"].value)}} }',
rowSeparator=''',
''',
suffix='''
] }''',
filterQuery='^F$',
filterColumn='gender')

{ "events" : [
    { "name" : "Melanie White", "purchase" : "iPhone" },
    { "name" : "Jean Griffith", "purchase" : "Power drill" },
    { "name" : "Melanie White", "purchase" : "iPad" }
] }

There is also an option to store the results in multiple files. Each file will contain the prefix, an processed row, and the suffix.

In [16]:
cli.templating(p2.project_id,
prefix='''{ "events" : [
''',
template='    { "name" : {{jsonize(cells["name"].value)}}, "purchase" : {{jsonize(cells["purchase"].value)}} }',
rowSeparator=''',
''',
suffix='''
] }''',
filterQuery='^F$',
filterColumn='gender',
output_file='advanced.json',
splitToFiles=True)

Export to files complete. Last file: advanced_3.json


Filenames are suffixed with the row number by default (e.g. `advanced_1.json`, `advanced_2.json` etc.). There is another option to use the value in the first column instead:

In [17]:
cli.templating(p2.project_id,
prefix='''{ "events" : [
''',
template='    { "name" : {{jsonize(cells["name"].value)}}, "purchase" : {{jsonize(cells["purchase"].value)}} }',
rowSeparator=''',
''',
suffix='''
] }''',
filterQuery='^F$',
filterColumn='gender',
output_file='advanced.json',
splitToFiles=True,
suffixById=True)

Export to files complete. Last file: advanced_melanie.white@example2.edu.json


Check the results in the current directory

In [18]:
os.listdir(os.getcwd())

['advanced_melanie.white@example2.edu.json',
 'duplicates-deletion.json',
 'advanced_1.json',
 'duplicates.csv',
 'advanced_2.json',
 'advanced_3.json',
 'advanced_jean.griffith@example5.org.json']

Because our project "advanced" contains duplicates in the first column "email" this command will overwrite files (e.g. `advanced_melanie.white@example2.edu.json`). When using this option, the first column should contain unique identifiers.

## Delete project

In [19]:
cli.delete(p2.project_id)

Project 2004697124065 has been successfully deleted


## Getting help

In [20]:
help(cli)

Help on module google.refine.cli in google.refine:

NAME
    google.refine.cli - Functions used by the command line interface (CLI)

FILE
    /srv/conda/envs/kernel/lib/python2.7/site-packages/google/refine/cli.py

FUNCTIONS
    apply(project_id, history_file)
        Apply OpenRefine history from json file to project.
    
    create(project_file, project_format=None, columnWidths=None, encoding=None, guessCellValueTypes=False, headerLines=None, ignoreLines=None, includeFileSources=False, limit=None, linesPerRow=None, processQuotes=True, projectName=None, projectTags=None, recordPath=None, separator=None, sheets=None, skipDataLines=None, storeBlankCellsAsNulls=True, storeBlankRows=True, storeEmptyStrings=True, trimStrings=False)
        Create a new project from file.
    
    delete(project_id)
        Delete project.
    
    download(url, output_file=None)
        Integrated download function for your convenience.
    
    export(project_id, encoding=None, output_file=None, export_

Client and server can be executed on different machines. Host and port of the OpenRefine server can be specified:

In [21]:
cli.refine.REFINE_HOST = 'localhost'
cli.refine.REFINE_PORT = '3333'

Please file an [issue](https://github.com/opencultureconsulting/openrefine-client/issues) if you miss some features in the command line interface or if you have tracked a bug. And you are welcome to ask any questions!